# Mnist_nn_softmax

## import

In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

## Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0
    
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

## Data load & Pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    
    # input할 때, 기본적으로 4개가 필요한데, Data load만 하면 channel이 '1'이어서 생략이 되므로 
    #  4종류의 output을 위해 expand-dims로 추가함 => expand_dims : channel을 한 개 더 만들어주는 역할 (-1:마지막 열에 추가)
    train_data = np.expand_dims(train_data, axis=-1)         # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)           # [N, 28, 28] -> [N, 28, 28, 1]
    
    train_data, test_data = normalize(train_data, test_data) # [0 ~ 255] -> [0 ~ 1]    // 나누기 255
    
    # One hot incoding : ex) 7 -=10개의 배열 중 7번째가'1'인 것
    train_labels = to_categorical(train_labels, 10)          # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10)            # [N,] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255
    test_data = test_data.astype(np.float32) / 255
    
    return train_data, test_data

## Performance Function

In [37]:
# 모델에 image를 어떤 숫자인지 logits로 확인 (해당 label과 softmax로 구한 logits를 비교하며 loss 구하기)
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))
    return loss

# Logits와 Label의 shape은 [batch size, label_dim]
def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))     # boolean 값과 숫자값으로 바꿔야 평균 계산이 가능
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)    

## Model function

In [38]:
# shape을 펼쳐주는 역할
def flatten():
    return tf.keras.layers.Flatten()

# label_dim : output으로 나가는 채널은 몇개?
def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

## Create model (class version)

In [39]:
# class 타입으로 Network 모델을 짤 때는 tf.keras.Model을 상속받음
class create_model(tf.keras.Model):
    def __init(self, label_dim):
        super(create_model_class, self).__init()
        weight_init = tf.keras.initializers.RandomNormal()   # 평균 0, 분산 1인 가우시안 적용
        
        self.model = tf.keras.Sequential()                   # list의 자료구조 타입 (층층이 쌓여가는 것이 list 축적
        self.model.add(flatten())                            # [N, 28, 28, 1] -> [N, 784]
        
        for i in range(2):
            # [N, 784] -> [N, 256] -> [N, 256] 으로 shape 변화
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())
            
        self.model.add(dense(label_dim, weight_init))
   
   # 이러한 model을 불렀을 때, 어떤 output을 만들어야 하는가
    def call(self, x, training = None, mask = None):
        x = self.model(x)
        
        return x   
        

## Create model (function version)

In [40]:
def create_model_function(label_dim):
    weight_init =tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256, weight_init))
        model.add(sigmoid())
        
    model.add(dense(label_dim, weight_init))
    
    return model

## Define data & hyper-parameter

In [41]:
# dataset
train_x, train_y, test_x, test_y=load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & Optimizer & writer

In [42]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir= 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

## Restore checkpoint & start train or test phase

In [ ]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn = network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
        
    # train phase
    with summary_writer.as_default():         # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars = zip(grads, network.variables))
                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    
                    test_accuracy = accuracy_fn(network, test_input, test_label)
                    
                    tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.3412, train_loss: 2.27224231, train_accuracy: 0.2969, test_Accuracy: 0.2458
Epoch: [ 0] [    1/  468] time: 1.9411, train_loss: 2.28239822, train_accuracy: 0.2422, test_Accuracy: 0.2050
Epoch: [ 0] [    2/  468] time: 2.2813, train_loss: 2.27772832, train_accuracy: 0.2031, test_Accuracy: 0.1992
Epoch: [ 0] [    3/  468] time: 2.6652, train_loss: 2.23038435, train_accuracy: 0.2031, test_Accuracy: 0.1999
Epoch: [ 0] [    4/  468] time: 3.0471, train_loss: 2.24921131, train_accuracy: 0.2578, test_Accuracy: 0.2259
Epoch: [ 0] [    5/  468] time: 3.4380, train_loss: 2.22847080, train_accuracy: 0.2891, test_Accuracy: 0.2726
Epoch: [ 0] [    6/  468] time: 3.8395, train_loss: 2.21989512, train_accuracy: 0.2344, test_Accuracy: 0.2209
Epoch: [ 0] [    7/  468] time: 4.2033, train_loss: 2.19239473, train_accuracy: 0.2188, test_Accuracy: 0.2083
Epoch: [ 0] [    8/  468] time: 4.5926,

Epoch: [ 0] [   74/  468] time: 35.6717, train_loss: 0.97761875, train_accuracy: 0.7500, test_Accuracy: 0.8071
Epoch: [ 0] [   75/  468] time: 36.1361, train_loss: 0.86542356, train_accuracy: 0.8516, test_Accuracy: 0.8143
Epoch: [ 0] [   76/  468] time: 36.7838, train_loss: 0.99626100, train_accuracy: 0.7656, test_Accuracy: 0.8161
Epoch: [ 0] [   77/  468] time: 37.2171, train_loss: 0.78286600, train_accuracy: 0.8594, test_Accuracy: 0.8135
Epoch: [ 0] [   78/  468] time: 37.5864, train_loss: 0.90307415, train_accuracy: 0.8359, test_Accuracy: 0.8145
Epoch: [ 0] [   79/  468] time: 37.9657, train_loss: 0.75771165, train_accuracy: 0.8516, test_Accuracy: 0.8126
Epoch: [ 0] [   80/  468] time: 38.7253, train_loss: 0.80108333, train_accuracy: 0.8281, test_Accuracy: 0.8087
Epoch: [ 0] [   81/  468] time: 39.1493, train_loss: 0.82294172, train_accuracy: 0.8125, test_Accuracy: 0.8040
Epoch: [ 0] [   82/  468] time: 39.5591, train_loss: 0.87961602, train_accuracy: 0.7734, test_Accuracy: 0.7970
E

[코드](https://github.com/deeplearningzerotoall/TensorFlow/blob/master/tf_2.x/lab-10-1-1-mnist_nn_softmax.ipynb)